In [57]:
from pyomo.environ import *
from pao.pyomo import *

### Example Setting: German government creates a procurement auction of fruits for Laim's community Kindergarten in Munich. The auction data is:
- Procurement_items = {Apples, Bananas, Tomatos}
- Participating_Qualifies_Suppliers = {Christina_GmbH, Lucas_GmbH}
- Auctioneer_Demand = {15,25,50}
- Auctioneer_Budget = {10,20,50} 
- Christina_Production_Cost = {18.75,12.5,37.5} 
- Lucas_Production_Cost = {7.5,15,50} 
- Lucas_Supplier_Capacity = {15,25,50} 
- Christina_Supplier_Capacity = {10,25,50} 

In [58]:
# Create a model object
model = ConcreteModel("Upper-level: Auction Problem")

In [59]:
# Subscripts
model.i = Set(initialize=['Apples', 'Bananas', 'Tomatos'], doc='Items')
model.j = Set(initialize=['Christina_GmbH', 'Lucas_GmbH'], doc='Suppliers')

In [60]:
# Upper-level Parameters
#   Parameters
#       D(i)  demand for item i
#         /    apples    15
#              bananas   25  /
#              tomatos   50
#       B(i)  budget for item i
#         /    apples    10
#              bananas   20  /
#              tomatos   50;
model.Demand = Param(model.i, initialize={'Apples':15, 'Bananas':25, 'Tomatos':50}, doc='Budget Items')
model.Budget = Param(model.i, initialize={'Apples':10, 'Bananas':20, 'Tomatos':50}, doc='Demand Items')

In [61]:
SupplierCapacity = {
    ('Christina_GmbH', 'Apples'): 15,
    ('Lucas_GmbH', 'Apples'): 15,
    ('Christina_GmbH', 'Bananas'): 25,
    ('Lucas_GmbH', 'Bananas'): 25,
    ('Christina_GmbH', 'Tomatos'):50,
    ('Lucas_GmbH', 'Tomatos'): 50,
}

In [62]:
model.CapacitySupply = Param(model.j, model.i, initialize=SupplierCapacity, doc='Supply Capacity of Suppliers')

In [63]:
ProductionCosts = {
    ('Christina_GmbH', 'Apples'): 18.75,
    ('Lucas_GmbH', 'Apples'): 7.5,
    ('Christina_GmbH', 'Bananas'): 12.5,
    ('Lucas_GmbH', 'Bananas'): 15,
    ('Christina_GmbH', 'Tomatos'): 37.5,
    ('Lucas_GmbH', 'Tomatos'): 50,
}

In [64]:
model.ProductionCosts = Param(model.j, model.i, initialize=ProductionCosts, doc='Production Cost per Supplier')

In [65]:
model.X = Var(model.j, model.i, domain=Binary, doc='Decision Variable X')
model.P = Var(model.j, model.i, bounds=(0, None), doc='Decision Variable P')

In [66]:
def auctionObjectiveFunction(model):
    return sum(model.P[j,i]*model.X[j,i] for j,i in model.j*model.i)

In [67]:
model.UpperLevelObjective = Objective(rule=auctionObjectiveFunction, sense=minimize, doc='Auction Problem')

In [68]:
def singleSourcing(model, i):
    return sum(model.X[j,i] for j in model.j) == 1
model.SorcingConstraint = Constraint(model.i, rule=singleSourcing, doc='There is at most 1 winner')

In [69]:
def budgetRequirement(model, i):
    return sum(model.ProductionCosts[j,i] * model.X[j,i] for j in model.j) <= model.Budget[i]
model.BudgetConstraint = Constraint(model.i, rule=budgetRequirement, doc='Auctioneer budget is respected')

In [70]:
def demandRequirement(model, i):
    return sum(model.CapacitySupply[j,i] * model.X[j,i] for j in model.j) >= model.Demand[i]
model.DemandConstraint = Constraint(model.i, rule=demandRequirement, doc='Auctioneer demand is fullfilled')

In [71]:
# def pricingObjectiveFunction(submodel):
#     return sum((model.P[j,i] - model.ProductionCosts[j,i])*model.X[j,i] for j,i in model.j*model.i)

In [82]:
model.L = SubModel(fixed=[model.X])

    'pao.pyomo.components.SubModel'>) on block 'Upper-level: Auction Problem'
    with a new Component (type=<class 'pao.pyomo.components.SubModel'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [83]:
model.L.o = Objective(rule=(sum((model.P[j,i] - model.ProductionCosts[j,i])*model.X[j,i] for j,i in model.j*model.i)), sense=maximize, doc='Pricing Problem')

In [84]:
model.pprint()

7 Set Declarations
    CapacitySupply_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    P_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    ProductionCosts_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    X_index : Size=1, Index=None, Or

In [87]:
'''Non-linear Solver (non-bilevel)'''
solver = SolverFactory('mindtpy').solve(model)

'''Linear Solver (bilevel)'''
# solver = Solver('pao.pyomo.MIBS').solve(model)

'Linear Solver (bilevel)'

In [88]:
model.pprint()

7 Set Declarations
    CapacitySupply_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    P_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    ProductionCosts_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    j*i :    6 : {('Christina_GmbH', 'Apples'), ('Christina_GmbH', 'Bananas'), ('Christina_GmbH', 'Tomatos'), ('Lucas_GmbH', 'Apples'), ('Lucas_GmbH', 'Bananas'), ('Lucas_GmbH', 'Tomatos')}
    X_index : Size=1, Index=None, Or